# Considerazioni su $ \sum_{n=1}^{\infty}\frac{1}{n^2} = \frac{\pi}{6} $

Calculate the following sum:  

$
\sum_{n=1}^\infty {1\over n^2}={\pi^2\over 6}
=\lim_{N\to\infty} S(N)
\quad
\text{with}
\quad
S(N)=\sum_{n=1}^N {1\over n^2}
$

I expect greater precision when adding larger numbers to smaller ones (a condition satisfied in the reverse sum).  
The reason is that if I have a number, say of the order of $2^m$, I know that in the interval $[2^m, 2^{m+1})$ there are exactly $\epsilon_{mach}$ numbers, and therefore the next floating-point number is at a distance of $2^m \cdot \epsilon_{mach}$. Thus, if I have a small number, the distance to the next floating-point number is small, and adding a larger number represents it better. If I have a large number, the distance to the next floating-point number is large, and therefore adding a smaller number is risky because I might not reach the next floating-point number.

In [ ]:
# import cell
using Plots
using LaTeXStrings
using Printf
function autosave(fig, filename)
    path = "C:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\relazione\\immagini"
    savefig(fig, joinpath(path, filename))
end
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\linear_systems.jl")

In [ ]:
#function cell

#The first two calculate the sum with single precision
#This one uses normal ordering
function sum_norm_ord32(N)
    sum = Float32(0)  
    for n in 1:N
        sum += Float32(1.0/n^2)
    end
    return sum
end

#this one uses reverse ordering
function sum_reverse_ord32(N)
    sum = Float32(0)
    #the intermediate number returns the step to take between N and 1.
    for n in N:-1:1
        sum += Float32(1.0/n^2)
    end
    return sum
end

#The second two calculate the sum with double precision
#This one uses normal ordering
function sum_norm_ord64(N)
    sum = Float64(0)  
    for n in 1:N
        sum += Float64(1.0/n^2)
    end
    return sum
end

#this one uses reverse ordering
function sum_reverse_ord64(N)
    sum = Float64(0)
    #the intermediate number returns the step to take between N and 1.
    for n in N:-1:1
        sum += Float64(1.0/n^2)
    end
    return sum
end

## Single precision
**(a)** Calculate the sum in single precision using normal ordering, $n=1,2,3,\ldots,N$.

**(b)** Calculate the sum in single precision using reverse ordering, $n=N,\ldots,2,1$.

In [ ]:
#Calculating the difference between the expected value of the sum and the calculated value of the sum
N = 8000

#Initializing the vector which contains the N of the sum
N_val = [i for i = 1:N]

#generating the array of pi^2/6
expected_val = fill((Float64(pi))^2/6, N)

#Calculating the sum with single precision
sum_n_val32 = sum_norm_ord32.(N_val)        #vector containing the sums with various N
sum_r_val32 = sum_reverse_ord32.(N_val)     #vector containing the sums with various N

diff_n = abs.(expected_val - sum_n_val32)
diff_r = abs.(expected_val - sum_r_val32)


**(c)** Study the convergence of both implementations as a function of $N$ by plotting $|S(N)-\pi^2/6|$.

There is a flattening after $n=4096=2^{12}$. Squared and inverted, it becomes $2^{-24}$, which is a number smaller than $\epsilon_{mach}$. For this reason, it is rounded to 0, and the sum does not improve.  
Moreover, before the flattening, the forward sum behaves like a straight line. This is due to rounding. It can also be seen that the reverse sum behaves like $\frac{1}{n}$, a result that can be obtained mathematically.

In [ ]:
#Plot cell
format_tick_scientific(x) = @sprintf("%.e", x)
fig = plot(xlabel=L"N", ylabel=L"\Delta = |S(N) - \pi^2/6|", 
     legend=:topright, 
     framestyle=:box, 
     grid=true, gridalpha=0.5,
     yformatter = format_tick_scientific,
     legendfontsize = 12,
     )

plot!(N_val, diff_n, 
      ylimits=(0.0, 0.001), 
      color=:navy,
      lw = 1.5,
      label=L"Somma\ diretta")

plot!(N_val, diff_r, 
      ylimits=(0.0, 0.001), 
      color=:darkorange,
      lw = 1.5,
      label=L"Somma\ inversa")

display(fig)
autosave(fig, "1211.pdf")

Il modello è $log(y) = -log(n)$

In [ ]:
#Interpolation of the two sum
D = hcat(log.(N_val))
y = log.(diff_n)
c1 = least_sq(D, y)
D = hcat(log.(N_val))
y = log.(diff_r)
c2 = least_sq(D, y)

display("Coefficients for the direct sum:")
display(c1[1])
display("Coefficients for the reverse sum:")
display(c2[1])

# Double precision
**(d)** Repeat (a)-(c) using double precision.

I verify that, despite significantly increasing the sum terms, I cannot observe the deviation between the two functions.

In [ ]:
M = 100000

#Initializing the vector which contains the M of the sum
M_val = [i for i = 1:M]

#generating the array of pi^2/6
expected_val_M = fill((Float64(pi))^2/6, M)

#Calculating the sum with double precision
sum_n_val64 = sum_norm_ord64.(M_val)        #vector containing the sums with various M
sum_r_val64 = sum_reverse_ord64.(M_val)     #vector containing the sums with various M

diff_n_M = abs.(expected_val_M - sum_n_val64)
diff_r_M = abs.(expected_val_M - sum_r_val64)

In [ ]:
#Plot cell
format_tick1(x) = @sprintf("%.6f", x)
format_tick2(x) = @sprintf("%.f", x)
format_tick_scientific1(x) = @sprintf("%.e", x)
format_tick_scientific2(x) = @sprintf("%.2e", x)
x_ticks = [i*10.0^4.0 for i in 4:1:11]
y_ticks = [i*10.0^(-5.0) for i in 1:0.25:5]

fig=plot(xlabel=L"M", ylabel=L"\Delta", 
    legend=:topright,
    framestyle=:box, 
    grid=true, gridalpha=0.5,
    xlimits=(50000, M-1000), 
    ylimits=(0.9*10.0^(-5.0), 2.2*10.0^(-5.0)), 
    xticks = x_ticks, yticks = y_ticks,
    xformatter = format_tick_scientific1, yformatter = format_tick_scientific2,
    legendfontsize = 12,
    )

plot!(fig, M_val, diff_n_M, 
      lw=6,
      color=:navy, 
      label=L"Somma\ diretta",
      )

plot!(fig, M_val, diff_r_M, 
      lw=2,
      color=:darkorange,
      label=L"Somma\ inversa",
      )

display(fig)
autosave(fig, "1212.pdf")